In [1]:
from builtins import print

import numpy as np
import cplex
import random
import pandas as pd
import math
import sys
from numpy import linalg as LA

# from sklearn.preprocessing.tests.test_base import toarray
from sphinx.addnodes import index

In [27]:
filenName = 'southeast-asia-copy.xlsx'
threshHold = 0
numberCandidates = 3

def readData(fileName,threshHold,numberCandidates,numberSkill):
#     dataset = pd.read_excel(fileName)
#     '''
#     R[i][j] is skill-depth of candidates i-th in skill j-th
#     '''
#     R = dataset.iloc[:5,1:numberSkill].values
# #     R = normalizingData(R)
# #     print(R)
#     nickNames = dataset.iloc[:3738,0].values
    # mean and standard deviation
    mu, sigma = 0, 0.1 
    R = np.random.normal(mu, sigma, (3738,38))
    R_before_normalize = R
    nickNames = []
    '''
    Sorting based on skill-depth by descending order
    '''
    R2 = -np.sort(-np.array(R),axis=0)
    '''
    E is sum of h(number of selected candidates) largest skill-depth
    '''
    E = []
    E_before_normalize = []
    '''
    skillScore is sum of all skill-depth of a candidates
    '''
    skillScore = np.sum(R,axis=0)
    ''''''
    for i in range(len(R2[0])):
        sum_skill_largest = 0
        for j in range(numberCandidates):
            sum_skill_largest += R2[j][i]
        E.append(sum_skill_largest)
        ##
    R2 = -np.sort(-np.array(R_before_normalize), axis=0)
    for i in range(len(R2[0])):
        sum_skill_largest = 0
        for j in range(numberCandidates):
            sum_skill_largest += R2[j][i]
        E_before_normalize.append(sum_skill_largest)
    tau = 0.001
    totalCandidates = len(R)
    z = [elem * 0.3 for elem in E]
    numberSkill = len(R[0])
    # z = np.zeros(numberSkill)
    # c = [random.randrange(1, 50, 1) for i in range(totalCandidates)]
    c = np.zeros(totalCandidates)
    C = cplex.infinity
    return (E,E_before_normalize,R,R_before_normalize,skillScore,nickNames,tau,z,c,C,numberSkill,totalCandidates)

def normalizingData(R):
    R_copy = np.copy(R)
    R_copy = np.sort(R_copy,axis=0)
    dictList = []
    for index in range(len(R_copy[0])):
        dict = {R_copy[0][index] : 1}
        lastValue = 1
        for index2 in range(1,len(R_copy)):
            if (R_copy[index2][index] not in dict):
                lastValue += 1
                dict[R_copy[index2][index]] = lastValue
        dictList.append(dict)
    for index in range(len(R[0])):
        for index2 in range(len(R)):
            R[index2][index] = dictList[index][R[index2][index]]
    return R

def compute_ObjectiveMDSB(X,numberSkill,E_before_normalize,R_before_normalize):
    sum = 0
    for index in range(numberSkill):
        tmp = 0
        for j in X:
            tmp+= R_before_normalize[j][index]
        sum += (E_before_normalize[index]-tmp) **2
    return sum

def compute_Objective(previousState,X,numberSkill,totalCandidates,tau,E,R):
    sum = 0
    indcies = [index for index in range(len(X)) if (X[index] != 0)]
    for j in range(numberSkill):
        tmp = 0
        for i in range(len(indcies)):
            tmp+=R[indcies[i]][j] * X[i]
        sum += (E[j] - tmp) * (E[j] - tmp)
    ##
    for index in indcies:
        sum -= tau * (2*previousState[index]-1) * X[index]
    # print(sum)
    return sum

In [28]:
class PMDSB:
    def __init__(self):
        self.model = cplex.Cplex()
    ##
    def buildModel(self , previous_step,E, R,tau,z,c,C,numberSkill,totalCandidates):
        self.model.objective.set_sense(self.model.objective.sense.minimize)
        self.model.parameters.optimalitytarget.set(1)
        '''
        Number of variables equals to number of candidates
        '''
        variables = []
        for i in range(totalCandidates):
            variables.append('x'+str(i))
        '''
        Add variables and set upper bounds & lower bounds
        lower bound of all variables is 0 
        upper bound of all variables is 1
        '''
        ub = np.ones(totalCandidates)
        lb = np.zeros(totalCandidates)
        self.model.variables.add(names=variables,lb=lb,ub=ub)
        # self.model.variables.add(names=variables,lb=lb,ub=ub)
        # self.model.variables.set_types([(elem, self.model.variables.type.continuous) for elem in variables])
        # self.model.variables.add(names=variables,types=['I']*totalCandidates)
        '''
        Set up model function, after decomposition
        Quadratics coefficient for candidate i-th will be: 
            R[i,1] ^2 + R[i,2] ^2 + ... + R[i,m] ^ 2
        '''
        # Calculating quadratic coefficients
#         quad_variables = []
#         for canIndex1 in range(totalCandidates):
#             for canIndex2 in range(canIndex1,totalCandidates):
#                 coefficients = 0
#                 for skill_Index in range(numberSkill):
#                     if (canIndex1 != canIndex2):
#                         coefficients += 2 * R[canIndex1][skill_Index] * R[canIndex2][skill_Index]
#                     else:
#                         coefficients += R[canIndex1][skill_Index] **2
#                 quad_variables.append((canIndex1,canIndex2,coefficients))
#         print(quad_variables)
#         self.model.objective.set_quadratic_coefficients(quad_variables)
        A = 2*R.dot(R.T)
#         print(A)
        row = [i for i in range(totalCandidates)]
        qmat = [[row, A[i]] for i in range(totalCandidates)]
        self.model.objective.set_quadratic(qmat)
        ## Linear Coefficients
        R_copy = np.copy(R)
        for i in range(len(R_copy[0])):
            R_copy[:,i] = [2*element * E[i] for element in R_copy[:,i]]
        linear_coefficients = np.sum(R_copy,axis=1)
        ##
        linear_coefficients = [(linear_coefficients[index] + tau * (2*previous_step[index] - 1)) * -1 for index in range(len(linear_coefficients))]
        linear_variables = []
        for i in range(totalCandidates):
            linear_variables.append((i, linear_coefficients[i]))
        #self.model.objective.set_linear(linear_variables)
        '''
        Setting ups constraints 
        1. SUM of all variables is number of candidates
        2. Minimum Skill Score 
        3. X[i] is either 0 or 1 .
        4. Constraint sum(c[i]*x[i]) <= C
        '''
        # Constraint #1
        linear_constraints = [
            cplex.SparsePair(ind=[index for index in range(totalCandidates)], val=[1 for i in range(totalCandidates)])]
        rhs_linear_constraints = [numberCandidates]
        senses = ['E']
        # Constraint #2
        '''
        senses = ["E", "L", "G", "R"]
        Each entry must be one of 'G', 'L', 'E', and 'R', indicating greater-than or equal, less-than or equal, equality, and ranged constraints,respectively.
        '''
        for i in range(numberSkill):
            row = [R[index, i] for index in range(totalCandidates)]
            linear_constraints.append(cplex.SparsePair(ind=variables.copy(), val=row))
            rhs_linear_constraints.append(z[i])
            senses.append('G')
        # for i in range(totalCandidates):
        #     # val.append(1)
        #     # val.append(1)
        #     # row.append(i)
        #     # row.append(i)
        #     linear_constraints.append(cplex.SparsePair(ind=[i], val=[1]))
        #     linear_constraints.append(cplex.SparsePair(ind=[i], val=[1]))
        #     rhs_linear_constraints.append(0)
        #     rhs_linear_constraints.append(1)
        #     senses.append('G')
        #     senses.append('L')


        self.model.linear_constraints.add(lin_expr=linear_constraints, rhs=rhs_linear_constraints, senses=senses)
        # constraints #3 x(x-1) = 0
        # for i in range(totalCandidates):
        #     linear_constraints_of_quadratics = cplex.SparsePair(ind=[variables[i]], val=[1])
        #     quad_constraints = cplex.SparseTriple(ind1=[variables[i]], ind2=[variables[i]], val=[-1])
        #     quad_sense = 'E'
        #     quad_contraints_rhs = 0
        #     self.model.quadratic_constraints.add(lin_expr=linear_constraints_of_quadratics,
        #                                          quad_expr=quad_constraints,
        #                                          sense=quad_sense,
        #                                          rhs=quad_contraints_rhs)
        # Constraint #4
        self.model.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind=variables.copy(), val=c)],
            senses=['L'],
            rhs=[C])

In [29]:
def solvingPMDSB(tauInput,number_Skill):
    # option 1 is MDSB
    # option 2 is PMDSB
    E,E_before_normalize,R,R_before_normalize, skillScore, nickNames, tau, z, c, C, numberSkill, totalCandidates = readData(fileName=filenName,
                                                                                            threshHold=threshHold,
                                                                                           numberCandidates=numberCandidates,
                                                                                           numberSkill=number_Skill)
    ## random pick
    tau = tauInput
    indices = [i for i in range(totalCandidates)]
    global X_0
    global isSolvedtotalCandidates
    global previousValue
    global objectValue
    '''
    Assign initial variables
    '''
    isSolved = True
    X_0 = np.zeros(totalCandidates)
    previousValue = sys.float_info.max
    '''
    '''
    for i in range(numberCandidates):
        rand_index = random.choice(indices)
        X_0[rand_index] = 1
        indices.remove(rand_index)
    ##
    epsilon = 0.00001
    executionTime = 0
    while True:
        modelPMDSB = PMDSB()
        modelPMDSB.buildModel(X_0,E=E, R=R,tau=tau,z= z,c= c,C= C,numberSkill= numberSkill,totalCandidates= totalCandidates)
        # cplex._internal._parameter_classes.optimalitytarget_constants = 3
        modelPMDSB.model.set_log_stream(None)
        modelPMDSB.model.set_error_stream(None)
        modelPMDSB.model.set_warning_stream(None)
        modelPMDSB.model.set_results_stream(None)
        # modelPMDSB.model.set_problem_type(modelPMDSB.model.problem_type.MIQP)
        startTime = modelPMDSB.model.get_time()
        print("Starting")
        modelPMDSB.model.solve()
        endTime = modelPMDSB.model.get_time()
        executionTime += (endTime-startTime)
        executionTime += (endTime-startTime)
        print("executionTime ",executionTime)
        # if (modelPMDSB.model.solution.status.infeasible):
        #     print("Infeasible")
        #     isSolved = False
        #     break
        X_1 = modelPMDSB.model.solution.get_values()
        currentValue = compute_Objective(X_0,X_1,numberSkill,totalCandidates,tau,E,R)
        objectValue = currentValue
        if (math.fabs(previousValue - currentValue) < epsilon):
            X_0 = X_1
            break
        X_0 = X_1
        previousValue = currentValue

    if (isSolved):
        print("Solution for problems using PMDSB is: ")
        X_0 = [0 - X_0[i] for i in range(len(X_0))]
        topValue = (np.sort(X_0))[:3].tolist()
        print(topValue)
        indcies = []
        ##
        for index in range(len(X_0)):
            if (X_0[index] in topValue):
                indcies.append(index)
                topValue.remove(X_0[index])
        ##
        print(indcies)
        # print([X_0[index] for index in range(len(X_0)) if X_0[index] != 0])
        print("Objective Value is: ")
        print(math.sqrt(compute_ObjectiveMDSB(indcies,numberSkill=numberSkill,E_before_normalize=E_before_normalize,R_before_normalize=R_before_normalize)))
        print("Number Skill: ")
        print(numberSkill)
        print("Execution Time: " , executionTime)

In [30]:
initialTau =0.001
# while (initialTau <= 10000):
#     solving(initialTau,3)
#     initialTau = initialTau * 10
#     print("Tau: ", initialTau)

Starting


CplexSolverError: CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.

In [7]:
# Starting
# executionTime  15.943049907684326
# Starting
# executionTime  17.696418285369873
# Starting
# executionTime  24.090574264526367
# Solution for problems using PMDSB is: 
# [-0.6723233391499864, -0.46424230157675334, -0.44684946342154486]
# [2, 110, 235]
# Objective Value is: 
# 1788131.0192036266
# Number Skill: 
# 37
# Execution Time:  24.090574264526367
# Tau:  0.01
# Starting
# executionTime  18.76253604888916
# Starting
# executionTime  33.692580223083496
# Starting
# executionTime  36.18282413482666
# Solution for problems using PMDSB is: 
# [-0.6723233391499864, -0.46424230157675334, -0.44684946342154486]
# [2, 110, 235]
# Objective Value is: 
# 1788131.0192036266
# Number Skill: 
# 37
# Execution Time:  36.18282413482666
# Tau:  0.1
# Starting
# executionTime  5.663553714752197
# Starting
# executionTime  15.106323719024658
# Starting
# executionTime  18.871869564056396
# Solution for problems using PMDSB is: 
# [-0.6723233391499864, -0.46424230157675334, -0.44684946342154486]
# [2, 110, 235]
# Objective Value is: 
# 1788131.0192036266
# Number Skill: 
# 37
# Execution Time:  18.871869564056396
# Tau:  1.0
# Starting
# executionTime  4.538817882537842
# Starting
# executionTime  18.418675899505615
# Starting
# executionTime  22.151966094970703
# Solution for problems using PMDSB is: 
# [-0.6723233391499864, -0.46424230157675334, -0.44684946342154486]
# [2, 110, 235]
# Objective Value is: 
# 1788131.0192036266
# Number Skill: 
# 37
# Execution Time:  22.151966094970703
# Tau:  10.0
# Starting
# executionTime  10.755067825317383
# Starting
# executionTime  21.302369594573975
# Starting
# executionTime  29.177839756011963
# Solution for problems using PMDSB is: 
# [-0.6723233391499864, -0.46424230157675334, -0.44684946342154486]
# [2, 110, 235]
# Objective Value is: 
# 1788131.0192036266
# Number Skill: 
# 37
# Execution Time:  29.177839756011963
# Tau:  100.0
# Starting
# executionTime  7.394812107086182
# Starting
# executionTime  18.287106037139893
# Starting
# executionTime  23.203173637390137
# Solution for problems using PMDSB is: 
# [-0.6723233391499864, -0.46424230157675334, -0.44684946342154486]
# [2, 110, 235]
# Objective Value is: 
# 1788131.0192036266
# Number Skill: 
# 37
# Execution Time:  23.203173637390137
# Tau:  1000.0
# Starting
# executionTime  17.033519744873047
# Starting
# executionTime  24.839051723480225
# Starting
# executionTime  30.965263843536377
# Solution for problems using PMDSB is: 
# [-0.6723233391499864, -0.46424230157675334, -0.44684946342154486]
# [2, 110, 235]
# Objective Value is: 
# 1788131.0192036266
# Number Skill: 
# 37
# Execution Time:  30.965263843536377
# Tau:  10000.0
# Starting
# executionTime  5.0216498374938965
# Starting
# executionTime  5.516232013702393
# Starting
# executionTime  7.8150739669799805
# Solution for problems using PMDSB is: 
# [-0.6723233391499864, -0.46424230157675334, -0.44684946342154486]
# [2, 110, 235]
# Objective Value is: 
# 1788131.0192036266
# Number Skill: 
# 37
# Execution Time:  7.8150739669799805
# Tau:  100000.0

In [35]:
class MDSB:
    def __init__(self):
        self.model = cplex.Cplex()
    def buildModel(self,E, R,tau,z,c,C,numberSkill,totalCandidates):
        self.model.objective.set_sense(self.model.objective.sense.minimize)
        variables = []
        for i in range(totalCandidates):
            variables.append('x' + str(i))
        '''
        Add variables and set upper bounds & lower bounds
        '''
        self.model.variables.add(names=variables)
        '''
        Set up variables types is binary
        ['C', 'I', 'B', 'S', 'N'] are integer, binary, semi_continuous, semi_integer
        '''
        self.model.variables.set_types([(elem,self.model.variables.type.binary) for elem in variables])
        '''
        Set up model function, after decomposition
        Quadratics coefficient for candidate i-th will be: 
            
        Linear coefficient for candidate i-th will be:
            -(2*(E[1] * R[i,1] + E[2] * R[i,2] + .... + E[m] * R[i,m]) + tau 
        '''
        ## Calculating quadratic coefficients
        # print("Quadratic Coefficients:")
#         quad_variables = []
#         for canIndex1 in range(totalCandidates):
#             for canIndex2 in range(canIndex1,totalCandidates):
#                 coefficients = 0
#                 for skill_Index in range(numberSkill):
#                     # print(f"{canIndex1}-{canIndex2} -{R[canIndex1][skill_Index]}-{R[canIndex2][skill_Index]}")
#                     if (canIndex1 != canIndex2):
#                         coefficients += 2*R[canIndex1][skill_Index] * R[canIndex2][skill_Index]
#                     else:
#                         coefficients += R[canIndex1][skill_Index] ** 2
#                 # print(f"{coefficients}")
#                 # quad_variables.append((canIndex1,canIndex2,coefficients))

#         self.model.objective.set_quadratic_coefficients(quad_variables)
        A = 2*R.dot(R.T)
#         print(A)
        row = [i for i in range(totalCandidates)]
        qmat = [[row, A[i]] for i in range(totalCandidates)]
        self.model.objective.set_quadratic(qmat)
        ## Linear Coefficients
        R_copy = np.copy(R)
        for i in range(len(R_copy[0])):
            R_copy[:, i] = [-2 * element * E[i] for element in R_copy[:, i]]
        linear_coefficients = np.sum(R_copy, axis=1)
        linear_variables = []
        for i in range(totalCandidates):
            linear_variables.append((i, linear_coefficients[i]))
        ##
        # print("Linear Coefficients")
        # print(linear_coefficients)
        self.model.objective.set_linear(linear_variables)
        '''
        Setting ups constraints 
        1. SUM of all variables is number of candidates
        2. Minimum Skill Score 
        3. X[i] is either 0 or 1 .
        4. Constraint sum(c[i]*x[i]) <= C
        '''
        # Constraint #1
        linear_constraints = [cplex.SparsePair(ind=[index for index in range(totalCandidates)],val=[1 for i in range(totalCandidates)])]
        rhs_linear_constraints = [numberCandidates]
        senses = ['E']
        # Constraint #2
        '''
        senses = ["E", "L", "G", "R"]
        Each entry must be one of 'G', 'L', 'E', and 'R', indicating greater-than or equal, less-than or equal, equality, and ranged constraints,respectively.
        '''
        for i in range(numberSkill):
            row = [R[index, i] for index in range(totalCandidates)]
            linear_constraints.append(cplex.SparsePair(ind=variables.copy(),val=row))
            rhs_linear_constraints.append(z[i])
            senses.append('G')

        self.model.linear_constraints.add(lin_expr=linear_constraints, rhs=rhs_linear_constraints, senses=senses)
        # Constraint #4
        self.model.linear_constraints.add(
            lin_expr=[cplex.SparsePair(ind=variables.copy(), val=c)],
            senses=['L'],
            rhs=[C])

In [36]:
def solving(option,number_Skill):
    if option == 1:
        E,E_before_normalize,R,R_before_normalize, skillScore, nickNames, tau, z, c, C, numberSkill, totalCandidates = readData(fileName=filenName,
                                                                                           threshHold=threshHold,
                                                                                           numberCandidates=numberCandidates,
                                                                                           numberSkill=number_Skill)
        modelMDSB = MDSB()
        modelMDSB.buildModel( E=E, R=R,tau=tau,z= z,c= c,C= C,numberSkill= numberSkill,totalCandidates= totalCandidates)
        modelMDSB.model.set_log_stream(None)
        modelMDSB.model.set_error_stream(None)
        modelMDSB.model.set_warning_stream(None)
        modelMDSB.model.set_results_stream(None)
        startTime = modelMDSB.model.get_time()
        modelMDSB.model.solve()
        endTime = modelMDSB.model.get_time()
        # if (modelMDSB.model.solution.status.infeasible):
        #     print("Infeasible")
        #     return
        print("Solution for problems using MDSB model is: ")
        solution = modelMDSB.model.solution.get_values()
        print([nickNames[element] for element in range(len(solution)) if (solution[element] != 0)])
        print("Objective Function Values is: ")
        result = modelMDSB.model.solution.get_objective_value()
        for index in range(numberSkill):
            result += E[index]**2
        # print(math.sqrt(result))
        # result = math.sqrt(result)
        # print(result)
        indecies = [element for element in range(len(solution)) if (solution[element] != 0)]
        print(indecies)
        print(math.sqrt(compute_ObjectiveMDSB(indecies,numberSkill,E,R)))
        # DuongsValue = [6,330,456]
        # print(math.sqrt(compute_ObjectiveMDSB(DuongsValue)))
        print("Time Execution")
        print(endTime-startTime)
        print("Number skill: ",numberSkill)

In [38]:
solving(1,38)
solving(initialTau,38)

CplexSolverError: CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.